In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

ens='cA211.530.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/test/data_merge/NST_a.h5_main'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

In [ ]:
flags={
    'cc2pt':True,
    'cc3pt':True,
    # 'l1l2':False,
}

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    t=data['2pt'][opa+'_'+opb] if opa+'_'+opb in data['2pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get2pt(opb,opa,set(diags)-{'N-pi0i',}))
    if opb+'_'+opa in data['2pt'].keys(): # contibution for c.c of T-pi0i
        t=data['2pt'][opb+'_'+opa]; diag='T-pi0f'
        res+= np.conj(t[diag]) if diag in t.keys() and diag in diags else 0
    return res
def get2ptMat(ops,diags=yu.diags_all):
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def get3pt(opa,opb,insert,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa1,opb,insert,diags)-np.sqrt(1/3)*get3pt(opa2,opb,insert,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get3pt(opa,opb1,insert,diags)-np.sqrt(1/3)*get3pt(opa,opb2,insert,diags)
    t=data['3pt'][opa+'_'+opb][insert] if opa+'_'+opb in data['3pt'] else {}
    res=np.sum([t[diag] for diag in t.keys() if diag in diags],axis=0)
    if yu.getNpars(opa,opb)==(2,1):
        res+=np.conj(get3pt(opb,opa,insert,set(diags)-{'N-jPi','N-j-pi0i'})[:,::-1])*(yu.gtCj[insert.split('_')[0]])
    return res
def get3ptMat(opas,opbs,insert,diags=yu.diags_all,NpiNpi=False):
    skipNpiNpi=lambda opa,opb: yu.getNpars(opa,opb)==(2,2) and not NpiNpi
    t=np.transpose([[get3pt(opa,opb,insert,diags) if not skipNpiNpi(opa,opb) else np.zeros([Ncfg,int(insert.split('_')[-1])+1])  for opb in opbs] for opa in opas],[2,3,0,1])
    if flags['cc3pt']:
        flags['cc3pt']=False
        tt=get3ptMat(opbs,opas,insert,diags,NpiNpi)[:,::-1]*(yu.gtCj[insert.split('_')[0]])
        flags['cc3pt']=True
        t=(t+np.conj(np.transpose(tt,[0,1,3,2])))/2
    return t

In [ ]:
def run2ptGEVP(pt,ofs,diags=yu.diags_all,title=None,ylim1=None,ylim2=None,compQ=False):
    fig, axs = yu.getFigAxs(1,2) if compQ==False else yu.getFigAxs(1,3)
    title=title if title=='None' else 'pt={}; ofs={}'.format(pt,ofs)
    fig.suptitle(title)
    
    Nop=len(ofs)
    tRef=1; tmax0=15
    tmin= 1-tRef if tRef<0 else tRef+1
    
    res={}
    
    for l,color,xshift in zip(['l1','l2'],['r','b'],[-0.1,0.1]):
        if l!='l1':
            continue
        ops=yu.getops(pt,l,ofs)
        dat=get2ptMat(ops,diags)
        
        irow=0; icol=0
        def func(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0)) if tRef>=0 else np.log(eVals)/tRef
            return En.T
        mean,err,cov=yu.jackknife(dat,func)
        res['E']=(mean,err)
        for ind in range(Nop):
            tmax=tmax0-3*ind
            tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','d','o'][ind])
            axs[irow,icol].set_xticks(range(0,tmax0,2))  
        axs[irow,icol].set_ylim(ylim1)
        
        irow=0; icol=1
        for j in range(1):
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
            mean,err,cov=yu.jackknife(dat,func)
            res['v']=(mean,err)
            for ind in range(Nop):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
                # print(tMean)
                # print(tErr)
            
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
                axs[irow,icol].set_xticks(range(0,tmax0,2))  
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
        
        if compQ:
            irow=0; icol=2
            def func(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs,comp)=yu.GEVP(t,tRef,compQ)
                t=np.abs(comp/(comp[:,range(Nop),range(Nop)])[:,:,None])
                return [t[:,0,-1],t[:,-1,0]]
            mean,err,cov=yu.jackknife(dat,func)
            for ind in range(2):
                tmax=tmax0
                tMean=mean[ind][tmin:tmax]; tErr=err[ind][tmin:tmax]
            
                axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=['s','<','>'][ind])
                axs[irow,icol].set_xticks(range(0,tmax0,2))  
            axs[irow,icol].set_ylim([0,1])
            # t=np.array([mean[0],err[0]])
            # print(t.T)
            # t=np.array([mean[1],err[1]])
            # print(t.T)
            # print()
    return res
            
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','n,pi+'),('N1pi1,a','p,pi0')],ylim1=[0.3,1.1])        
# run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1],compQ=True)  # t0~5
# run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1],compQ=True)   # t0~3

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def run3pt2ptGEVP(ptofas,ptofbs,insert,prefactor,proj,diags=yu.diags_all,title=None,ylim1=None):
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    res={}
    
    prefactor=1 if prefactor=='re' else -1j if prefactor=='im' else prefactor
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    needsVEV= pta==ptb and 'N-j' in diags and insert in ['id_j+','g5_j-'] and proj in ['P0','Pz']
    
    proj=np.array([[1,0],[0,1]])/2 if proj=='P0' else np.array([[1,0],[0,-1]])/2 if proj=='Pz' else proj
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
        
        def t_get(tf):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf) for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j'][insert]]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            if needsVEV:
                t2pt_sub=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)*proj[0,0]+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska)*proj[1,1])
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            t=[(
                np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                -(t2pt_sub[tf]*tVEVj[0] if needsVEV else 0)
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            )
            for i,tf in enumerate(tfs)]
        
            t=yu.prefactorDeep(t,prefactor)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)
        res[case]=(mean,err)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
            # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
        
    return res

In [ ]:
tfs=[10,12,14]; tRef=1
def get_eVec(eVecs):
    t=eVecs[:,0,0]
    t=eVecs[:,:,0]/t[:,None]
    # return t[10]
    return np.mean(t[10:14+1,:],axis=0)

def runFF(ptofas,ptofbs,FF,diags=yu.diags_all,title=None,ylim1=None):
    
    res={}
    
    mN=lat.getaEN(0); EN=lat.getaEN(1); qk=(2*math.pi)/(lat.L/lat.a); K=np.sqrt(2*mN**2/(EN*(EN+mN)))
    t_5=-(2*mN)/(K*qk)
    t_A=1j*(2*mN)/(K*(EN+mN))
    t_Pt=(4*mN**2)/(K*qk*(mN-EN)); t_Pt2=(2*mN)/(mN-EN)
    t_Pi=-1j*(4*mN**2)/(K*qk**2); t_Pi2=-(2*mN*(EN+mN))/(qk**2)
    
    pips={
        'GS':[(1*lat.amu*lat.aInv,'id_j+','P0')],
        'G5':[(t_5*lat.ZS,'g5_j-','Pz')],
        'GA0':[(1j*lat.ZA,'g5gz_j-','Pz')],
        'GA':[(t_A*(1/2)*lat.ZA,'g5gx_j-','Px'),(t_A*(1/2)*lat.ZA,'g5gy_j-','Py')],
        'GPt':[(t_Pt2*t_A*(-1/2)*lat.ZA,'g5gx_j-','Px'),(t_Pt2*t_A*(-1/2)*lat.ZA,'g5gy_j-','Py'),(t_Pt*lat.ZA,'g5gt_j-','Pz')],
        'GPi':[(t_Pi2*t_A*(-1/2)*lat.ZA,'g5gx_j-','Px'),(t_Pi2*t_A*(-1/2)*lat.ZA,'g5gy_j-','Py'),(t_Pi*lat.ZA,'g5gz_j-','Pz')],
    }[FF]
    
    fig, axs = yu.getFigAxs(1,1)
    # title=title if title=='None' else 'opa={}; opb={}; insert={}; prefactor={}'.format(opa,opb,insert,prefactor)
    fig.suptitle(title)
    
    pta,ofas=ptofas; ptb,ofbs=ptofbs
    needsVEV= pta==ptb and 'N-j' in diags and FF in ['GS'] 
    
    proj2mat={
        'P0':np.array([[1,0],[0,1]])/2,
        'Px':np.array([[0,1],[1,0]])/2,
        'Py':np.array([[0,-1j],[1j,0]])/2,
        'Pz':np.array([[1,0],[0,-1]])/2,
    }
    
    for case in ['noGEVP','GEVP']:
        pta,ofas=ptofas; ptb,ofbs=ptofbs
        if case=='noGEVP':
            ofas=[ofas[0]]; ofbs=[ofbs[0]]
            
        Nparas=[yu.getNpar(op) for op in yu.getops(pta,'l1',ofas)]
        Nparbs=[yu.getNpar(op) for op in yu.getops(ptb,'l1',ofbs)]
        maska=np.array([[0 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        maskb=np.array([[0 if l==r==2 else 1 for l in Nparbs] for r in Nparbs])
        # maska=np.array([[1 if l==r==2 else 1 for l in Nparas] for r in Nparas])
        # maskb=np.array([[1 if l==r==2 else 1 for l in Nparbs] for r in Nparbs]) 
        
        def t_get(tf,t_insert):
            t=[[get3ptMat(yu.getops(pta,la,ofas),yu.getops(ptb,lb,ofbs),t_insert+'_'+str(tf),diags)[:,1:tf]\
                for lb in ['l1','l2']] for la in ['l1','l2']]
            return np.transpose(t,[2,3,4,5,0,1]) # cfg,tc,opa,opb,la,lb
        
        dat3pt=[t_get(tf,insert) for prefactor,insert,proj in pips for tf in tfs]
        dat2pt=[get2ptMat(yu.getops(pta,'l1',ofas),yu.diags_all),get2ptMat(yu.getops(pta,'l2',ofas),yu.diags_all),
                get2ptMat(yu.getops(ptb,'l1',ofbs),yu.diags_all),get2ptMat(yu.getops(ptb,'l2',ofbs),yu.diags_all),]
        datVEVj=[data['VEV']['j']['id_j+']]
        dat=[dat3pt,dat2pt,datVEVj]
        
        AInv=yu.getCoeMat(pta); BInv=yu.getCoeMat(ptb)

        def func(dat):
            t=yu.meanDeep(dat)
            t3pt,t2pt,tVEVj=t
            
            (eVals_al1,eVecs_al1)=yu.GEVP(t2pt[0],tRef)
            (eVals_al2,eVecs_al2)=yu.GEVP(t2pt[1],tRef)
            (eVals_bl1,eVecs_bl1)=yu.GEVP(t2pt[2],tRef)
            (eVals_bl2,eVecs_bl2)=yu.GEVP(t2pt[3],tRef)
            
            eVec_al1=get_eVec(eVecs_al1)
            eVec_al2=get_eVec(eVecs_al2)
            eVec_bl1=get_eVec(eVecs_bl1)
            eVec_bl2=get_eVec(eVecs_bl2)
            
            t2pt_a=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            t2pt_b=(np.einsum('i,tij,j,ij->t',np.conj(eVec_bl1),t2pt[2],eVec_bl1,maskb)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_bl2),t2pt[3],eVec_bl2,maskb))/2
            
            if needsVEV:
                t2pt_sub=(np.einsum('i,tij,j,ij->t',np.conj(eVec_al1),t2pt[0],eVec_al1,maska)+
                    np.einsum('i,tij,j,ij->t',np.conj(eVec_al2),t2pt[1],eVec_al2,maska))/2
            
            eVec_a=np.array([eVec_al1,eVec_al2])
            eVec_b=np.array([eVec_bl1,eVec_bl2])
            
            def t_get2(i):
                res=0
                for i_pip,pip in enumerate(pips):
                    prefactor,insert,proj=pip
                    proj=proj2mat[proj]
                    res+=prefactor*np.einsum('tijlm,ba,li,mj,al,bm->t',t3pt[i_pip*len(tfs)+i],proj,np.conj(eVec_a),eVec_b,AInv,np.conj(BInv)) # ij for ops; l,m for la,lb; ab for diracs
                return res
            
            t=[(
                t_get2(i)
                -(t2pt_sub[tf]*tVEVj[0]*pips[0][0] if needsVEV else 0)
            )/np.sqrt(
                np.real(t2pt_a[tf])*np.real(t2pt_b[tf])*\
                np.real(t2pt_a[1:tf][::-1])/np.real(t2pt_a[1:tf])*\
                np.real(t2pt_b[1:tf])/np.real(t2pt_b[1:tf][::-1])
            )
            for i,tf in enumerate(tfs)]
        
            t=yu.prefactorDeep(t,1)
            return t
        
        (mean,err,cov)=yu.jackknife(dat,func)
        res[case]=(mean,err)

        irow=0; icol=0
        for i_tf,tf in enumerate(tfs):
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
            # axs[irow,icol].errorbar(np.arange(1,tf),tMean,tErr,color='b' if case=='noGEVP' else 'r',fmt=['s','d','o'][i_tf],mfc='white' if case=='noGEVP' else None)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
    axs[0,0].axis('auto')
    
    return res

In [ ]:
res={}
res['2ptGEVP_0']=run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1])
res['2ptGEVP_1']=run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1],compQ=True)


diags=yu.diags_jLoopless
res['GS']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'GS',diags=diags)
res['GA0']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'GA0',diags=diags)

res['G5']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'G5',diags=diags)
res['GA']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'GA',diags=diags)
res['GPt']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'GPt',diags=diags)
res['GPi']=runFF(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'GPi',diags=diags)

import pickle
with open('Lattice2023_cA24.pkl','wb') as f:
    pickle.dump(res,f)

In [ ]:
# res={}
# res['2ptGEVP_0']=run2ptGEVP('0,0,0',[('a','p'),('N1pi1,a','12')],ylim1=[0.3,1.1])
# res['2ptGEVP_1']=run2ptGEVP('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12')],ylim1=[0.3,1.1],compQ=True)
# res['ratio_id+_0']=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'id_j+',1,'P0',diags=yu.diags_jLoopless)
# res['ratio_g5gz-_0']=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,0',[('a','p'),('N1pi1,a','12')]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-1.64,-1.48])
# res['ratio_g5gt-_1']=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gt_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.2,0.5])
# res['ratio_g5-_1']=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5_j-',1,'Pz',diags=yu.diags_jLoopless,ylim1=[-10,-2.5])
# res['ratio_g5gz-_1']=run3pt2ptGEVP(('0,0,0',[('a','p'),('N1pi1,a','12')]),('0,0,1',[('a','p'),('N1pi0,a','12'),('N0pi1,a','12'),]),'g5gz_j-',-1j,'Pz',diags=yu.diags_jLoopless,ylim1=[-0.8,-0.2])

# import pickle
# with open('Lattice2023_cA24.pkl','wb') as f:
#     pickle.dump(res,f)